In [21]:
import time

from pyautocad import Autocad, APoint
from shapely.geometry.point import Point

In [24]:
class AutocadMLeaderWorker:
    def __init__(self):
        self.acad = Autocad()
        self.model = acad.model
        self.MLeader = None
        self.base_point = None
        
    def init_MLeader(self, layer):
        for obj in acad.iter_objects(['AcDbMLeader']):
            if obj.Layer == layer:
                self.MLeader = obj
                self.base_point = APoint([*obj.GetLeaderLineVertices(0)[:2], 0])
    
    def insert_MLeader(self, insert_data) -> None:
        if self.MLeader is None:
            raise ValueError('MLeader object is None')
        for layer, text, x, y in insert_data:
            MLeader_past = self.MLeader.Copy()
            MLeader_past.TextString = text
            MLeader_past.Layer = layer
            MLeader_past.Move(self.base_point, APoint([x, y, 0]))


In [25]:
MLeaderWorker = AutocadMLeaderWorker()
MLeaderWorker.init_MLeader('template')
data = [
    ('ВЫНОСКИ', '1', 3300.78, 1550.62),
    ('ВЫНОСКИ', '2', 3200.78, 1640.62),
    ('ВЫНОСКИ', '3', 3100.78, 1600.62)
]
MLeaderWorker.insert_MLeader(data)


## EZDXF

In [75]:
import ezdxf
from ezdxf.entities import MultiLeader

path = r'C:\Projects\TaxationTool\test_data\example_1_topographic_plan.dxf'
doc = ezdxf.readfile(path)
msp = doc.modelspace()

In [76]:
from ezdxf.math import Vec2

mleader: MultiLeader = msp.query('MULTILEADER[layer=="template"]').first
ml_builder = msp.add_multileader_mtext()
ml_builder.quick_leader('537', Vec2(3300.78, 1550.62), Vec2.from_deg_angle(45, 3))


AttributeError: 'MLeaderStyleCollection' object has no attribute 'set_mtext_style'

In [74]:
doc.save()